# Code to impute emissions rates, then output a file that specifies emissions per location per MWh of generation
The first challenge will be to get emissions rates (lb/mwh).  To do this I'll be taking EIA data on net generation and EPA data on emissions to calculate rates.  There will be many missings that I will fill in by taking capacity-weighted technology-planning_year means.

Once I have emissions rates, the goal will be to output a series of shapefiles of location-specific emissions per MWh. A challenge here is assigning output to to specific plants.  I assign production based on capacity.

### Brief Code Overview
1. Data cleaning (formatting columns, names, etc)
2. Merge EIA and EPA data into existing_gen_units using the EPA-EIA crosswalk. 
3. Impute generation for generators with missing generation.  To do this, I take plant-level generation and distribute that generation to generators based on each generator's capacity.
4. Calculate generation-weighted average emissions rates, then use those to fill in generators with missing emissions rates (missings get filled in with a technology-planningyear weighted mean).
5. Calculate aggregate emissions by multiplying rates by model-output specified generation.  As mentioned earlier, I calculate generator level generation either by calculating a generator's share of total capacity or net generation for that cluster-planningyear.
6. Split the emissions output by planning year and write to shapefiles.


## 1. Data cleaning

In [1]:
globals().clear()

import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import geopandas as gpd
import fiona
import math
from shapely.geometry import Point

C:\Users\lbeatty\AppData\Local\Temp\ipykernel_25164\1677484318.py:7: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
os.chdir('C:/Users/lbeatty/Documents/Lauren_MIP_Contribution/')


#PG existing gen units output
existing_gen_units_2030 = pd.read_csv('MIP_results_comparison/case_settings/26-zone/usensys-inputs-short/usensys-inputs-short/2030/base_short_2030/extra_outputs/existing_gen_units.csv')
existing_gen_units_2040 = pd.read_csv('MIP_results_comparison/case_settings/26-zone/usensys-inputs-short/usensys-inputs-short/2040/base_short_2040/extra_outputs/existing_gen_units.csv')
existing_gen_units_2050 = pd.read_csv('MIP_results_comparison/case_settings/26-zone/usensys-inputs-short/usensys-inputs-short/2050/base_short_2050/extra_outputs/existing_gen_units.csv')

#PG generators data
generators_data_2030 = pd.read_csv('MIP_results_comparison/case_settings/26-zone/usensys-inputs-short/usensys-inputs-short/2030/base_short_2030/Inputs/Generators_data.csv')
generators_data_2040 = pd.read_csv('MIP_results_comparison/case_settings/26-zone/usensys-inputs-short/usensys-inputs-short/2030/base_short_2030/Inputs/Generators_data.csv')
generators_data_2050 = pd.read_csv('MIP_results_comparison/case_settings/26-zone/usensys-inputs-short/usensys-inputs-short/2030/base_short_2030/Inputs/Generators_data.csv')

#filter by retirement year
#the code was written when existing_gen_units varied by year so lots of artifacts of the way the old output looked
existing_gen_units_2030 = existing_gen_units_2030.query('retirement_year>=2030')
existing_gen_units_2040 = existing_gen_units_2030.query('retirement_year>=2040')
existing_gen_units_2050 = existing_gen_units_2030.query('retirement_year>=2050')


#EIA-EPA Crosswalk
crosswalk = pd.read_csv("Data/epa_eia_crosswalk.csv")

# EIA 860 for Generator Info
eia860 = pd.read_excel("Data/eia860/3_1_Generator_Y2020.xlsx", skiprows=1)

# EIA923 for Generation Info
eia923_fuels = pd.read_excel("Data/eia923/EIA923_Schedules_2_3_4_5_M_12_2020_Final_Revision.xlsx", sheet_name='Page 1 Generation and Fuel Data', skiprows=5)
eia923_generators = pd.read_excel("Data/eia923/EIA923_Schedules_2_3_4_5_M_12_2020_Final_Revision.xlsx", sheet_name='Page 4 Generator Data', skiprows=5)

#Emissions
emissions = pd.read_csv("Data/CAMD/facilities_emissions.csv")
#only want year 2020
emissions = emissions.query('year==2020')

# PM25
pm25 = pd.read_excel("Data/eGRID2020 DRAFT PM Emissions.xlsx", sheet_name="2020 PM Unit-level Data", skiprows=1)


C:\Users\lbeatty\AppData\Local\Temp\ipykernel_25164\15306011.py:5: DtypeWarning: Columns (17,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  existing_gen_units_2030 = pd.read_csv('MIP_results_comparison/case_settings/26-zone/usensys-inputs-short/usensys-inputs-short/2030/base_short_2030/extra_outputs/existing_gen_units.csv')
C:\Users\lbeatty\AppData\Local\Temp\ipykernel_25164\15306011.py:6: DtypeWarning: Columns (17,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  existing_gen_units_2040 = pd.read_csv('MIP_results_comparison/case_settings/26-zone/usensys-inputs-short/usensys-inputs-short/2040/base_short_2040/extra_outputs/existing_gen_units.csv')
C:\Users\lbeatty\AppData\Local\Temp\ipykernel_25164\15306011.py:7: DtypeWarning: Columns (17,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  existing_gen_units_2050 = pd.read_csv('MIP_results_comparison/case_settings/26-zone/usensys-inputs-s

In [3]:
#############################
## Read in and format NEI ####
###############################

#nei data gives values for ammonia, vocs, and also pm2.5
#It also has so2 and nox output that mostly matches camd data
#I'll prioritise camd data, but fill in any missings with nei

#cems noncems just distinguish whether emissions were continuously monitored or imputed
#doesn't really matter for our purposes
nei_cems = pd.read_csv('Data/NEI/egucems_SmokeFlatFile_2020NEI_POINT_20230128_27mar2023_v1.csv', skiprows=16)
nei_noncems = pd.read_csv('Data/NEI/egunoncems_SmokeFlatFile_2020NEI_POINT_20230128_27mar2023_v0.csv', skiprows=15)
nei = pd.concat([nei_cems, nei_noncems])

#pull out the relevant pollutants
nei = nei.groupby(['oris_facility_code', 'oris_boiler_id', 'poll']).agg({'ann_value': 'sum'}).reset_index()
nei = nei[nei['poll'].isin(['CO', 'NH3', 'NOX', 'PM25-PRI', 'SO2', 'SO4', 'VOC'])]

#put weights in pounds
nei['poll']=nei['poll']+'_nei_tons'
nei['ann_value']=nei['ann_value']
nei = nei.pivot(index=['oris_facility_code', 'oris_boiler_id'], columns='poll', values='ann_value').reset_index()
nei = nei.rename(columns = {'oris_facility_code': 'CAMD_PLANT_ID', 'oris_boiler_id': 'CAMD_UNIT_ID'})



C:\Users\lbeatty\AppData\Local\Temp\ipykernel_25164\2972420735.py:11: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  nei_cems = pd.read_csv('Data/NEI/egucems_SmokeFlatFile_2020NEI_POINT_20230128_27mar2023_v1.csv', skiprows=16)
C:\Users\lbeatty\AppData\Local\Temp\ipykernel_25164\2972420735.py:12: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  nei_noncems = pd.read_csv('Data/NEI/egunoncems_SmokeFlatFile_2020NEI_POINT_20230128_27mar2023_v0.csv', skiprows=15)


In [4]:
#####################
## Format columns ###
#####################

# existing_gen_units
existing_gen_units_2030['plant_id_eia']=existing_gen_units_2030['plant_id_eia'].astype(int)
existing_gen_units_2040['plant_id_eia']=existing_gen_units_2040['plant_id_eia'].astype(int)
existing_gen_units_2050['plant_id_eia']=existing_gen_units_2050['plant_id_eia'].astype(int)

existing_gen_units_2030['generator_id']=existing_gen_units_2030['generator_id'].astype(str)
existing_gen_units_2040['generator_id']=existing_gen_units_2040['generator_id'].astype(str)
existing_gen_units_2050['generator_id']=existing_gen_units_2050['generator_id'].astype(str)

existing_gen_units_2030['planning_year']=2030
existing_gen_units_2040['planning_year']=2040
existing_gen_units_2050['planning_year']=2050

#bind all into one
existing_gen_units = pd.concat([existing_gen_units_2030, existing_gen_units_2040, existing_gen_units_2050])
existing_gen_units = existing_gen_units.rename(columns={'generator_id': 'EIA_GENERATOR_ID', 'plant_id_eia':'EIA_PLANT_ID'})

## EPA-EIA crosswalk
crosswalk = crosswalk[['CAMD_PLANT_ID', 'CAMD_UNIT_ID', 'CAMD_GENERATOR_ID', 'EIA_PLANT_ID', 'EIA_GENERATOR_ID', 'EIA_UNIT_TYPE']]
crosswalk['EIA_GENERATOR_ID'] = crosswalk['EIA_GENERATOR_ID'].astype(str)

##############
## EIA Data###
##############

# filter out AK and HI
eia860 = eia860[~eia860['State'].isin(['HI', 'AK'])]
eia923_fuels = eia923_fuels[~eia923_fuels['Plant State'].isin(['HI', 'AK'])]
eia923_generators = eia923_generators[~eia923_generators['Plant State'].isin(['HI', 'AK'])]


#rename columns and change formats
eia860 = eia860[['Plant Code', 'Generator ID', 'Nameplate Capacity (MW)', 'Planned Retirement Year', 'Planned Retirement Month', 'Synchronized to Transmission Grid', 'Technology']]
eia860.columns = ['EIA_PLANT_ID', 'EIA_GENERATOR_ID', 'Capacity', 'RetirementYear', 'RetirementMonth', 'SynchronizedToGrid', 'Technology']
eia860['EIA_GENERATOR_ID']=eia860['EIA_GENERATOR_ID'].astype(str)

eia923_fuels = eia923_fuels[['Plant Id', 'Plant Name', 'Plant State', 'Net Generation\n(Megawatthours)']]
eia923_fuels.columns = ['EIA_PLANT_ID', 'EIA_PLANT_NAME', 'EIA_STATE', 'NET_GEN_PLANT']
eia923_fuels = eia923_fuels.groupby('EIA_PLANT_ID').agg({'NET_GEN_PLANT': 'sum'}).reset_index()

eia923_generators = eia923_generators[['Plant Id', 'Generator Id', 'Net Generation\nYear To Date']]
eia923_generators.columns = ['EIA_PLANT_ID', 'EIA_GENERATOR_ID', 'NET_GEN_GENERATOR']
eia923_generators['EIA_GENERATOR_ID'] = eia923_generators['EIA_GENERATOR_ID'].astype(str)

##############
##EPA data####
##############
pm25['UNITID'] = pm25['UNITID'].astype(str)
pm25 = pm25.groupby(['ORISPL', 'UNITID']).agg({'PM25AN': 'sum'}).reset_index()
pm25.columns = ['facilityId', 'unitId', 'pm25']
pm25['unitId']=pm25['unitId'].astype(str)
pm25['facilityId']=pm25['facilityId'].astype(int)

emissions['unitId']=emissions['unitId'].astype(str)
emissions['facilityId']=emissions['facilityId'].astype(int)

emissions = pd.merge(emissions, pm25, on=['facilityId', 'unitId'], how='left')
emissions['nox_tons'] = emissions['noxMass']
emissions['so2_tons'] = emissions['so2Mass']
emissions['pm25_tons'] = emissions['pm25']

#rename columns
emissions = emissions.rename(columns = {'facilityId': 'CAMD_PLANT_ID', 'unitId': 'CAMD_UNIT_ID'})

C:\Users\lbeatty\AppData\Local\Temp\ipykernel_25164\3096755666.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  existing_gen_units_2040['plant_id_eia']=existing_gen_units_2040['plant_id_eia'].astype(int)
C:\Users\lbeatty\AppData\Local\Temp\ipykernel_25164\3096755666.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  existing_gen_units_2050['plant_id_eia']=existing_gen_units_2050['plant_id_eia'].astype(int)
C:\Users\lbeatty\AppData\Local\Temp\ipykernel_25164\3096755666.py:11: SettingWithCopyWarning: 
A v

## 2. Merge EIA and EPA data into existing_gen_units using the EPA-EIA crosswalk

In [5]:
#join existing_gen_units with crosswalk
existing_gen_units = pd.merge(existing_gen_units, crosswalk, on=['EIA_GENERATOR_ID', 'EIA_PLANT_ID'],how='left')

#in existing_gen_units, there's a fair amount of missing capacities that are available from eia data so I'm going to join in 
#eia860 and then fill in nans
existing_gen_units = pd.merge(existing_gen_units, eia860[['EIA_GENERATOR_ID', 'EIA_PLANT_ID', 'Capacity']], on=['EIA_GENERATOR_ID', 'EIA_PLANT_ID'], how='left')

#need net gen
existing_gen_units = pd.merge(existing_gen_units, eia923_fuels, on=['EIA_PLANT_ID'], how='left')
existing_gen_units = pd.merge(existing_gen_units, eia923_generators, on=['EIA_PLANT_ID', 'EIA_GENERATOR_ID'], how='left')

#lastly, need emissions
existing_gen_units = pd.merge(existing_gen_units, emissions, on=['CAMD_PLANT_ID', 'CAMD_UNIT_ID'], how='left')

#nei emissions
existing_gen_units = pd.merge(existing_gen_units, nei, on=['CAMD_PLANT_ID', 'CAMD_UNIT_ID'], how='left')


In [6]:
#fill in missing capacities
existing_gen_units['capacity_mw']=existing_gen_units['capacity_mw'].combine_first(existing_gen_units['Capacity'])

## 3. Impute generation for generators with missing generation.  To do this, I take plant-level generation and distribute that generation to generators based on each generator's capacity.

I'm going to calculate plant-level 'missing generation'
Then divy up the missing generation to generators according to capacity in MW

In [7]:
summed_plant_gen = existing_gen_units.groupby(['EIA_PLANT_ID', 'planning_year']).agg({'NET_GEN_GENERATOR': 'sum'}).reset_index()
summed_plant_gen.columns=['EIA_PLANT_ID', 'planning_year', 'sum_generator_gen']

existing_gen_units = pd.merge(existing_gen_units, summed_plant_gen, on=['EIA_PLANT_ID', 'planning_year'], how='left')

existing_gen_units['missing_generator_generation']=existing_gen_units['NET_GEN_GENERATOR'].isna().astype(int)
existing_gen_units['missing_generation'] = existing_gen_units['NET_GEN_PLANT'] - existing_gen_units['sum_generator_gen']


In [8]:
missing_generators = existing_gen_units.query('missing_generator_generation==1').\
    groupby(['EIA_PLANT_ID', 'planning_year']).\
    agg({'capacity_mw': 'sum'}).\
    reset_index()
missing_generators.columns=['EIA_PLANT_ID', 'planning_year', 'missing_generator_capacity']

existing_gen_units = pd.merge(existing_gen_units, missing_generators, on=['EIA_PLANT_ID', 'planning_year'], how='left')

In [9]:
## Make imputation

existing_gen_units['pct_missing_capacity'] = (existing_gen_units['capacity_mw']/existing_gen_units['missing_generator_capacity'])*existing_gen_units['missing_generator_generation']
existing_gen_units['imputed_net_gen'] = existing_gen_units['pct_missing_capacity']*existing_gen_units['missing_generation']


#replace missings with imputed
existing_gen_units['NET_GEN_GENERATOR'] = existing_gen_units['NET_GEN_GENERATOR'].combine_first(existing_gen_units['imputed_net_gen'])

## 4. Calculate generation-weighted average emissions rates, then use those to fill in generators with missing emissions rates.


In [10]:
#looks like there's broad agreement between camd and nei
#fill in missing camd data with nei data

existing_gen_units['nox_tons'] = existing_gen_units['nox_tons'].combine_first(existing_gen_units['NOX_nei_tons'])
existing_gen_units['so2_tons'] = existing_gen_units['so2_tons'].combine_first(existing_gen_units['SO2_nei_tons'])
existing_gen_units['pm25_tons'] = existing_gen_units['pm25_tons'].combine_first(existing_gen_units['PM25-PRI_nei_tons'])

#get rates
existing_gen_units['nox_rate'] = existing_gen_units['nox_tons']/existing_gen_units['NET_GEN_GENERATOR']
existing_gen_units['so2_rate'] = existing_gen_units['so2_tons']/existing_gen_units['NET_GEN_GENERATOR']
existing_gen_units['pm25_rate'] = existing_gen_units['pm25_tons']/existing_gen_units['NET_GEN_GENERATOR']
existing_gen_units['nh3_rate'] = existing_gen_units['NH3_nei_tons']/existing_gen_units['NET_GEN_GENERATOR']
existing_gen_units['voc_rate'] = existing_gen_units['VOC_nei_tons']/existing_gen_units['NET_GEN_GENERATOR']



#for now I'm going to omit units with negative net_gen since it doesn't make sense for them to have negative 'rates'
#I'll replace the rates with sample weighted-means
existing_gen_units.loc[existing_gen_units['nox_rate']<0, 'nox_rate']=np.nan
existing_gen_units.loc[existing_gen_units['pm25_rate']<0, 'pm25_rate']=np.nan
existing_gen_units.loc[existing_gen_units['so2_rate']<0, 'so2_rate']=np.nan
existing_gen_units.loc[existing_gen_units['nh3_rate']<0, 'nh3_rate']=np.nan
existing_gen_units.loc[existing_gen_units['voc_rate']<0, 'voc_rate']=np.nan

existing_gen_units.loc[existing_gen_units['nox_rate'].isin([np.inf]), 'nox_rate']=np.nan
existing_gen_units.loc[existing_gen_units['pm25_rate'].isin([np.inf]), 'pm25_rate']=np.nan
existing_gen_units.loc[existing_gen_units['so2_rate'].isin([np.inf]), 'so2_rate']=np.nan
existing_gen_units.loc[existing_gen_units['nh3_rate'].isin([np.inf]), 'nh3_rate']=np.nan
existing_gen_units.loc[existing_gen_units['voc_rate'].isin([np.inf]), 'voc_rate']=np.nan

In [11]:
############
## Calculate weighted average emissions rates by technology-planning year

# Define a function to calculate weighted average handling NaN values

def weighted_average(df):
    weighted_avgs = {}
    for col in df.columns:
        if '_rate' in col:  # Consider columns containing '_rate'
            df_valid = df.dropna(subset=[col, 'NET_GEN_GENERATOR'])
            if len(df_valid) == 0 or df_valid['NET_GEN_GENERATOR'].sum() == 0:
                weighted_avgs[col] = np.nan  # Return NaN if all weights in the group are zero
            else:
                weighted_avgs[col] = np.average(df_valid[col], weights=df_valid['NET_GEN_GENERATOR'])
    return pd.Series(weighted_avgs)

technology_rates = existing_gen_units.groupby(['technology', 'planning_year']).apply(weighted_average).reset_index()


In [12]:
#Ok last thing to do is fill in missings

technology_rates = technology_rates[['technology', 'planning_year', 'nox_rate', 'pm25_rate', 'so2_rate', 'voc_rate', 'nh3_rate']]
technology_rates.columns = ['technology', 'planning_year', 'noxrate_imputed', 'pm25rate_imputed', 'so2rate_imputed', 'vocrate_imputed', 'nh3rate_imputed']

existing_gen_units = pd.merge(existing_gen_units, technology_rates, on=['technology', 'planning_year'], how='left')

In [13]:
existing_gen_units['noxrate_imputed'] = existing_gen_units['nox_rate'].combine_first(existing_gen_units['noxrate_imputed'])
existing_gen_units['so2rate_imputed'] = existing_gen_units['so2_rate'].combine_first(existing_gen_units['so2rate_imputed'])
existing_gen_units['pm25rate_imputed'] = existing_gen_units['pm25_rate'].combine_first(existing_gen_units['pm25rate_imputed'])
existing_gen_units['vocrate_imputed'] = existing_gen_units['voc_rate'].combine_first(existing_gen_units['vocrate_imputed'])
existing_gen_units['nh3rate_imputed'] = existing_gen_units['nh3_rate'].combine_first(existing_gen_units['nh3rate_imputed'])

In [14]:
###############
## Little detour here to calculate year 2020 emissions
##This will serve as a gut-check and I'll output it so that in the results you can see the decline from 2020 --> 2030
## First I'll just take nei and camd data and sum it.
## Then I'll take observed net gen and multiply it by my imputed emissions rates
## Hopefully emissions will be close

reported_emissions_2020 = pd.merge(emissions, nei, how='outer', on=['CAMD_UNIT_ID', 'CAMD_PLANT_ID']).reset_index()
reported_emissions_2020['nox_tons'] = reported_emissions_2020['nox_tons'].combine_first(existing_gen_units['NOX_nei_tons'])
reported_emissions_2020['pm25_tons'] = reported_emissions_2020['pm25_tons'].combine_first(existing_gen_units['PM25-PRI_nei_tons'])
reported_emissions_2020['so2_tons'] = reported_emissions_2020['so2_tons'].combine_first(existing_gen_units['SO2_nei_tons'])
print(reported_emissions_2020[['nox_tons', 'pm25_tons', 'so2_tons', 'NH3_nei_tons', 'VOC_nei_tons']].sum())

test_emissions_2020 = pd.merge(eia860[['EIA_GENERATOR_ID', 'EIA_PLANT_ID', 'Capacity', 'Technology']], eia923_generators, on=['EIA_GENERATOR_ID', 'EIA_PLANT_ID'], how='left')
test_emissions_2020 = pd.merge(test_emissions_2020, eia923_fuels, on=['EIA_PLANT_ID'])
pct_capacity_2020 = test_emissions_2020.groupby(['EIA_PLANT_ID']).agg({'Capacity':'sum'}).reset_index()
pct_capacity_2020 = pct_capacity_2020.rename(columns={'Capacity':'sum_plant_capacity'})
test_emissions_2020 = pd.merge(test_emissions_2020, pct_capacity_2020, how='left', on='EIA_PLANT_ID')
test_emissions_2020['pct_capacity'] = test_emissions_2020['Capacity']/test_emissions_2020['sum_plant_capacity']
test_emissions_2020['predicted_gen']=test_emissions_2020['pct_capacity']*test_emissions_2020['NET_GEN_PLANT']

test_emissions_2020 = test_emissions_2020.rename(columns={'Technology':'technology'})
test_emissions_2020 = pd.merge(test_emissions_2020, technology_rates[technology_rates['planning_year']==2030], how='left', on=['technology'])

#replace the few negative net gens with zero
test_emissions_2020.loc[test_emissions_2020['predicted_gen']<0, 'predicted_gen']=0
test_emissions_2020 = test_emissions_2020[test_emissions_2020['predicted_gen']>0]

test_emissions_2020['nox_predicted']=test_emissions_2020['predicted_gen']*test_emissions_2020['noxrate_imputed']
test_emissions_2020['so2_predicted']=test_emissions_2020['predicted_gen']*test_emissions_2020['so2rate_imputed']
test_emissions_2020['pm25_predicted']=test_emissions_2020['predicted_gen']*test_emissions_2020['pm25rate_imputed']
test_emissions_2020['voc_predicted']=test_emissions_2020['predicted_gen']*test_emissions_2020['vocrate_imputed']
test_emissions_2020['nh3_predicted']=test_emissions_2020['predicted_gen']*test_emissions_2020['nh3rate_imputed']


test_emissions_2020 = test_emissions_2020[(test_emissions_2020['nox_predicted'] != 0) | (test_emissions_2020['so2_predicted'] != 0) | (test_emissions_2020['pm25_predicted'] != 0) | (test_emissions_2020['voc_predicted'] != 0) | (test_emissions_2020['nh3_predicted'] != 0)]
#drop if all predicted emissions are 0
print(test_emissions_2020[['nox_predicted', 'so2_predicted', 'pm25_predicted', 'voc_predicted', 'nh3_predicted']].sum())

#looks pretty close to me
#nh3 looks different but I think there's lots of missings
#test_emissions_2020 is actually what I'm going to bind into emissions and analyze in InMap


nox_tons        764117.084000
pm25_tons        88446.692134
so2_tons        808305.497843
NH3_nei_tons     18829.463402
VOC_nei_tons     20387.986583
dtype: float64
nox_predicted     787127.484352
so2_predicted     742088.506697
pm25_predicted    100743.871080
voc_predicted      24143.026328
nh3_predicted      34291.854018
dtype: float64


## 5. Calculate aggregate emissions by multiplying rates by model-output specified generation.  As mentioned earlier, I calculate generator level generation either by calculating a generator's share of total capacity or net generation for that cluster-planningyear.

Start by specifying which model, scenario, and column you want outputs for.

In [15]:
column = 'capacity_mw'

In [16]:
technology_year_total = existing_gen_units.groupby(['Resource', 'planning_year']).agg({column : 'sum'}).reset_index()
technology_year_total.columns = ['Resource', 'planning_year', 'technology_total']

#merge in total capacity or total net gen
existing_gen_units = pd.merge(existing_gen_units, technology_year_total, on=['Resource', 'planning_year'], how='left')


existing_gen_units['pct_total']=existing_gen_units[column]/existing_gen_units['technology_total']

existing_gen_units['nox_predicted']=existing_gen_units['pct_total']*existing_gen_units['noxrate_imputed']
existing_gen_units['so2_predicted']=existing_gen_units['pct_total']*existing_gen_units['so2rate_imputed']
existing_gen_units['pm25_predicted']=existing_gen_units['pct_total']*existing_gen_units['pm25rate_imputed']
existing_gen_units['voc_predicted']=existing_gen_units['pct_total']*existing_gen_units['vocrate_imputed']
existing_gen_units['nh3_predicted']=existing_gen_units['pct_total']*existing_gen_units['nh3rate_imputed']


In [17]:
#merge in plant locations
plants = pd.read_excel('Data/eia860/2___Plant_Y2017.xlsx', skiprows=1)
plants = plants[['Plant Code', 'Longitude', 'Latitude']]
plants.columns = ['EIA_PLANT_ID', 'Longitude', 'Latitude']

existing_gen_units = pd.merge(existing_gen_units, plants, on=['EIA_PLANT_ID'], how='left')

### Now think about emissions from new sources
All new natural gas plants show up in generation files with technology as '_naturalgas_' verus '_natural_gas'
Challenge here is to determine how much they emit, and *where*

First I make some plots that show that for most pollutants, emissions per unit of energy have gone down a lot in the past thirty years, but that progress really started petering out between 5-10 years ago.  Therefore, I'm going to take technology averages **for the last five years (2016-2020)**

Then what about where?  In the ex-post analysis code, I put new facilities at retired sites first (up to previous capacity), then equally distribute capacity to retired/nonretired sites based on capacity.  In this new world where everything is endogenized, this doesn't really make sense since capacity is endogenous!  Moreover in typical scenarios, you'll see a lot more new capacity than retired capacity so it doesn't really make sense to put, say 200MW of capacity at a site that used to have a 25MW capacity.

This actually makes things easier though. For all new capacity, I'll just distribute it based on capacity to *all* sites (operating and retired).

In [18]:
## Calculate emissions rates for new sources
generators_data_2030['planning_year']=2030
generators_data_2040['planning_year']=2040
generators_data_2050['planning_year']=2050
generators_data = pd.concat([generators_data_2030, generators_data_2040, generators_data_2050])


## Calculate rates for new sources
existing_gen_units['operating_date']=pd.to_datetime(existing_gen_units['operating_date'])
technology_rates_newsources = existing_gen_units[existing_gen_units['operating_date']>=pd.to_datetime('2016-01-01')].groupby(['technology', 'planning_year']).apply(weighted_average).reset_index()

#Add in CCS
# It looks like all new NGCT has heat rate of 6.36 and all new CCCCS has heat rate of 7.16
# For now, I'm just going to run with this
appendrows = technology_rates_newsources[technology_rates_newsources['technology']=='Natural Gas Fired Combined Cycle']
appendrows['technology'] = 'Natural Gas Fired Combined Cycle With CCS'

ccspenalty = 7.16/6.36

ratecolumns=['nox_rate', 'so2_rate', 'pm25_rate', 'nh3_rate', 'voc_rate']
appendrows[ratecolumns]=appendrows[ratecolumns].apply(lambda x: x * ccspenalty)
technology_rates_newsources = pd.concat([technology_rates_newsources, appendrows])


C:\Users\lbeatty\AppData\Local\Temp\ipykernel_25164\4141580381.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  appendrows['technology'] = 'Natural Gas Fired Combined Cycle With CCS'
C:\Users\lbeatty\AppData\Local\Temp\ipykernel_25164\4141580381.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  appendrows[ratecolumns]=appendrows[ratecolumns].apply(lambda x: x * ccspenalty)


In [19]:
### Now deal with new generation
#Isolate retired sites -- dates and 
new_sites = eia860.dropna(subset=["Technology"])
new_sites = new_sites[new_sites['Technology'].str.contains('Natural Gas')]
new_sites = pd.merge(new_sites, plants, how='left', on='EIA_PLANT_ID')
ipm_regions = gpd.read_file('Data/IPM_Regions/national_emm_boundaries.shp')

In [20]:
#make retired sites a geopandas
geometry = [Point(lon, lat) for lon, lat in zip(new_sites['Longitude'], new_sites['Latitude'])]
new_sites = gpd.GeoDataFrame(new_sites, geometry=geometry, crs='EPSG:4326')

In [21]:
#ipm_regions['region'] = ipm_regions['IPM_Region'].map({region: val for val, regions in cost_multiplier_region_map.items() for region in regions})
ipm_regions = ipm_regions.to_crs('EPSG:4326')

new_sites = gpd.sjoin(new_sites, ipm_regions, how="left", op="intersects")

c:\Users\lbeatty\AppData\Local\miniforge3\envs\powergenome\lib\site-packages\IPython\core\interactiveshell.py:3490: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [22]:
new_sites_df = pd.DataFrame(new_sites)
new_sites_df = new_sites_df.dropna(subset=['model_regi'])

# look for plants matched with multiple model regions
# For now I'm just going to take the top row, which will lead to a couple inconsistencies
#hopefully the shapefiles will become in better shape
new_sites_df = new_sites_df.groupby(['EIA_GENERATOR_ID', 'EIA_PLANT_ID']).first().reset_index()

# #want to distribute new capacity evenly to retired sites
# #be agnostic about type of natural gas plant -- eg new cc capacity can go at an old ct site
new_sites_total= new_sites_df.groupby(['model_regi']).agg({'Capacity':'sum'}).reset_index()
new_sites_total.columns = ['model_regi', 'Tot_Capacity']

new_sites_df = pd.merge(new_sites_df, new_sites_total, how='left', on=['model_regi'])
new_sites_df['pct_total']=new_sites_df['Capacity']/new_sites_df['Tot_Capacity']

new_sites_df = new_sites_df[['pct_total', 'Longitude', 'Latitude']]
newgenerators = generators_data[generators_data['Resource'].str.contains('naturalgas_')]
newgenerators.loc[newgenerators['Resource'].str.contains('_ccavg'), 'technology']= 'Natural Gas Fired Combined Cycle'
newgenerators.loc[newgenerators['Resource'].str.contains('_ctavg'), 'technology']= 'Natural Gas Fired Combustion Turbine'
newgenerators.loc[newgenerators['Resource'].str.contains('_ccccsavg'), 'technology']= 'Natural Gas Fired Combined Cycle With CCS'

newgenerators = pd.merge(newgenerators, technology_rates_newsources, how='left', on=['technology', 'planning_year'])
newgenerators = newgenerators.merge(new_sites_df, how="cross")

newgenerators['nox_predicted']=newgenerators['pct_total']*newgenerators['nox_rate']
newgenerators['so2_predicted']=newgenerators['pct_total']*newgenerators['so2_rate']
newgenerators['pm25_predicted']=newgenerators['pct_total']*newgenerators['pm25_rate']
newgenerators['voc_predicted']=newgenerators['pct_total']*newgenerators['voc_rate']
newgenerators['nh3_predicted']=newgenerators['pct_total']*newgenerators['nh3_rate']

newgenerators = newgenerators[['Longitude', 'Latitude', 'Resource', 'planning_year', 'nox_predicted', 'so2_predicted', 'pm25_predicted', 'voc_predicted', 'nh3_predicted']]

In [23]:
#bind all emissions sources together

existing_gen_units = existing_gen_units[['Longitude', 'Latitude', 'Resource', 'planning_year', 'nox_predicted', 'so2_predicted', 'pm25_predicted', 'voc_predicted', 'nh3_predicted']]
emissions=pd.concat([existing_gen_units, newgenerators], ignore_index=True)

print(emissions.shape[0])
#aggregate emissions in same spot/cluster
emissions=emissions.groupby(['Longitude', 'Latitude', 'Resource', 'planning_year']).agg({'nox_predicted':'sum', 'so2_predicted':'sum', 'pm25_predicted':'sum', 'voc_predicted':'sum', 'nh3_predicted':'sum'}).reset_index()
print(emissions.shape[0])


1307826
401978


## 6. Split the emissions output by planning year and write to shapefiles.

In [25]:
#####
## 2030
#####
emissions_2030 = emissions.query('planning_year==2030')

emissions_2030 = emissions_2030[['Longitude', 'Latitude', 'nox_predicted', 'so2_predicted', 'pm25_predicted', 'voc_predicted', 'nh3_predicted', 'Resource']]
emissions_2030.columns = ['Longitude', 'Latitude', 'NOx', 'SOx', 'PM2_5', 'VOC', 'NH3', 'Resource']

emissions_2030 = gpd.GeoDataFrame(
    emissions_2030, geometry = gpd.points_from_xy(emissions_2030.Longitude, emissions_2030.Latitude), crs='EPSG:4326')

emissions_2030 = emissions_2030.dropna().reset_index(drop=True)

emissions_2030.to_file(filename='InMap/MIP_Emissions/marginal_emissions_2030.shp')

#####
## 2040
#####

emissions_2040 = emissions.query('planning_year==2040')

emissions_2040 = emissions_2040[['Longitude', 'Latitude', 'nox_predicted', 'so2_predicted', 'pm25_predicted', 'voc_predicted', 'nh3_predicted','Resource']]
emissions_2040.columns = ['Longitude', 'Latitude', 'NOx', 'SOx', 'PM2_5', 'VOC', 'NH3', 'Resource']

emissions_2040 = gpd.GeoDataFrame(
    emissions_2040, geometry = gpd.points_from_xy(emissions_2040.Longitude, emissions_2040.Latitude), crs='EPSG:4326')

emissions_2040 = emissions_2040.dropna().reset_index(drop=True)

emissions_2040.to_file(filename='InMap/MIP_Emissions/marginal_emissions_2040.shp')

#####
## 2050
#####

emissions_2050 = emissions.query('planning_year==2050')

emissions_2050 = emissions_2050[['Longitude', 'Latitude', 'nox_predicted', 'so2_predicted', 'pm25_predicted', 'voc_predicted', 'nh3_predicted', 'Resource']]
emissions_2050.columns = ['Longitude', 'Latitude', 'NOx', 'SOx', 'PM2_5', 'VOC', 'NH3', 'Resource']

emissions_2050 = gpd.GeoDataFrame(
    emissions_2050, geometry = gpd.points_from_xy(emissions_2050.Longitude, emissions_2050.Latitude), crs='EPSG:4326')

emissions_2050 = emissions_2050.dropna().reset_index(drop=True)


emissions_2050.to_file(filename='InMap/MIP_Emissions/marginal_emissions_2050.shp')

